In [2]:
!pip install nbastats

In [1]:
import nbastats.nbastats as nbastats

In [8]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns

In [265]:
shots = shot_log("2014-15",'203128')
chart = nbastats.ShotChart('203128',season="2014-15").shotchart()
print shots.columns
print chart.columns

Index([u'CLOSEST_DEFENDER', u'CLOSEST_DEFENDER_PLAYER_ID', u'CLOSE_DEF_DIST', u'DRIBBLES', u'FGM', u'FINAL_MARGIN', u'GAME_CLOCK', u'GAME_ID', u'LOCATION', u'MATCHUP', u'PERIOD', u'PTS', u'PTS_TYPE', u'SHOT_CLOCK', u'SHOT_DIST', u'SHOT_NUMBER', u'SHOT_RESULT', u'TOUCH_TIME', u'W'], dtype='object')
Index([u'GRID_TYPE', u'GAME_ID', u'GAME_EVENT_ID', u'PLAYER_ID', u'PLAYER_NAME', u'TEAM_ID', u'TEAM_NAME', u'PERIOD', u'MINUTES_REMAINING', u'SECONDS_REMAINING', u'EVENT_TYPE', u'ACTION_TYPE', u'SHOT_TYPE', u'SHOT_ZONE_BASIC', u'SHOT_ZONE_AREA', u'SHOT_ZONE_RANGE', u'SHOT_DISTANCE', u'LOC_X', u'LOC_Y', u'SHOT_ATTEMPTED_FLAG', u'SHOT_MADE_FLAG'], dtype='object')


In [301]:
shots = pd.read_csv("all_chart_2014.csv")
shots.head()

,Unnamed: 0,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG
0,0,Shot Chart Detail,21400010,107,203112,Quincy Acy,1610612752,New York Knicks,1,1,22,Made Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),16-24 ft.,18,185,15,1,1
1,1,Shot Chart Detail,21400010,112,203112,Quincy Acy,1610612752,New York Knicks,1,0,31,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,16,-23,159,1,0
2,2,Shot Chart Detail,21400010,396,203112,Quincy Acy,1610612752,New York Knicks,4,10,26,Made Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,17,-71,162,1,1
3,3,Shot Chart Detail,21400010,436,203112,Quincy Acy,1610612752,New York Knicks,4,6,32,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,11,-97,60,1,0
4,4,Shot Chart Detail,21400010,454,203112,Quincy Acy,1610612752,New York Knicks,4,5,55,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,19,-144,129,1,0


In [327]:
all_shots = pd.read_csv("all_shots_2014.csv")
all_chart = pd.read_csv("all_chart_2014.csv")
players=set(all_shots['PLAYER_ID'])
joined_shots=pd.DataFrame()
joined_shots_1=pd.DataFrame()
for player in players:
    shots = all_shots.loc[all_shots.PLAYER_ID==player].reset_index(drop=True)
    chart = all_chart.loc[all_chart.PLAYER_ID==player].reset_index(drop=True)
    split_time = shots.GAME_CLOCK.str.split(':')
    col_names=[u'CLOSEST_DEFENDER', u'CLOSEST_DEFENDER_PLAYER_ID', u'CLOSE_DEF_DIST', 
               u'DRIBBLES', u'FGM', u'FINAL_MARGIN', u'GAME_CLOCK', u'GAME_ID', u'LOCATION', 
               u'MATCHUP', u'PERIOD', u'PTS', u'PTS_TYPE', u'SHOT_CLOCK', u'SHOT_DIST', 
               u'SHOT_NUMBER', u'SHOT_RESULT', u'TOUCH_TIME', u'W',u'GRID_TYPE',u'GAME_EVENT_ID',
               u'PLAYER_ID', u'PLAYER_NAME', u'TEAM_ID',u'MINUTES_REMAINING', u'SECONDS_REMAINING', 
               u'EVENT_TYPE', u'ACTION_TYPE', u'SHOT_TYPE', u'SHOT_ZONE_BASIC', 
               u'SHOT_ZONE_AREA', u'SHOT_ZONE_RANGE', u'SHOT_DISTANCE', u'LOC_X', u'LOC_Y', 
               u'SHOT_ATTEMPTED_FLAG', u'SHOT_MADE_FLAG']
    time = pd.DataFrame(shots.GAME_CLOCK.str.split(':').tolist(), columns="MIN SECONDS".split())
    shots['MIN'] = time['MIN']
    shots['SECONDS'] = time['SECONDS']
    shots['TIME']=60*shots.MIN.astype(int)+shots.SECONDS.astype(int)
    chart['TIME']=60*chart.MINUTES_REMAINING+chart.SECONDS_REMAINING
    games = set(chart["GAME_ID"])
    for game in games:
        game_shots_chart=chart.loc[chart.GAME_ID==game].reset_index(drop=True)
        game_shots_log = shots.loc[shots.GAME_ID==game].reset_index(drop=True)
        if game_shots_chart.shape[0]==game_shots_log.shape[0]:
            huh = pd.concat([game_shots_chart,game_shots_log],axis=1)
            huh=huh[col_names]
            joined_shots=joined_shots.append(huh)
        else:
            quarters=set(game_shots_chart['PERIOD'])
            for quarter in quarters:
                same_period=game_shots_log.loc[(game_shots_log.PERIOD==row['PERIOD'])].reset_index(drop=True)
                same_period_chart=game_shots_chart.loc[(game_shots_chart.PERIOD==row['PERIOD'])].reset_index(drop=True)
                if (not same_period.empty) and (not same_period_chart.empty):
                    if same_period_chart.shape[0]==same_period.shape[0]:
                        huh = pd.concat([same_period_chart,same_period],axis=1)
                        huh=huh[col_names]
                        joined_shots=joined_shots.append(huh)
                    else:
                        for index,row in same_period_chart.iterrows():
                            same_period['DIF']=abs(same_period.TIME-row.TIME)
                            df=same_period.sort(['DIF'],ascending=True)
                            same_time=same_period.loc[[0]]
                            if same_time.DIF[0]<5:
                                print "hi"
                                a=same_period_chart.loc[[index]].reset_index(drop=True)
                                huh=pd.concat([a,same_time],axis=1)
                                huh=huh[col_names]
                                joined_shots_1=joined_shots_1.append(huh,ignore_index=True)
    print player
joined_shots

202779
200745
hi
hi
hi
hi
hi
hi
hi
hi
200746
200748
200751
hi
hi
hi
hi
hi
hi
hi
hi
200752


KeyboardInterrupt: 

In [326]:
joined_shots.to_csv("joined_shots_2014.csv")
print joined_shots.shape,all_shots.shape,all_chart.shape

(200532, 40) (203590, 21) (205550, 22)


In [328]:
joined_shots_1


,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,DRIBBLES,FGM,FINAL_MARGIN,GAME_CLOCK,GAME_ID,GAME_ID,LOCATION,MATCHUP,PERIOD,PERIOD,PTS,PTS_TYPE,SHOT_CLOCK,SHOT_DIST,SHOT_NUMBER,SHOT_RESULT,TOUCH_TIME,W,GRID_TYPE,GAME_EVENT_ID,PLAYER_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG
0,"Andersen, Chris",2365,4.7,1,0,-4,6:25,21401007,21401007,A,"MAR 18, 2015 - POR @ MIA",4,4,0,2,1.6,20.9,18,missed,0.9,L,Shot Chart Detail,434,200746,200746,LaMarcus Aldridge,1610612757,6,21,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side Center(RC),16-24 ft.,19,122,154,1,0
1,"Andersen, Chris",2365,4.7,1,0,-4,6:25,21401007,21401007,A,"MAR 18, 2015 - POR @ MIA",4,4,0,2,1.6,20.9,18,missed,0.9,L,Shot Chart Detail,434,200746,200746,LaMarcus Aldridge,1610612757,6,21,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side Center(RC),16-24 ft.,19,122,154,1,0
2,"Andersen, Chris",2365,4.7,1,0,-4,6:25,21401007,21401007,A,"MAR 18, 2015 - POR @ MIA",4,4,0,2,1.6,20.9,18,missed,0.9,L,Shot Chart Detail,434,200746,200746,LaMarcus Aldridge,1610612757,6,21,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side Center(RC),16-24 ft.,19,122,154,1,0
3,"Andersen, Chris",2365,4.7,1,0,-4,6:25,21401007,21401007,A,"MAR 18, 2015 - POR @ MIA",4,4,0,2,1.6,20.9,18,missed,0.9,L,Shot Chart Detail,434,200746,200746,LaMarcus Aldridge,1610612757,6,21,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side Center(RC),16-24 ft.,19,122,154,1,0
4,"Millsap, Paul",200794,2.8,4,1,-8,7:17,21400501,21400501,H,"JAN 03, 2015 - POR vs. ATL",4,4,2,2,10.4,8.4,19,made,3.9,L,Shot Chart Detail,444,200746,200746,LaMarcus Aldridge,1610612757,7,14,Made Shot,Hook Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,36,36,1,1
5,"Millsap, Paul",200794,2.8,4,1,-8,7:17,21400501,21400501,H,"JAN 03, 2015 - POR vs. ATL",4,4,2,2,10.4,8.4,19,made,3.9,L,Shot Chart Detail,444,200746,200746,LaMarcus Aldridge,1610612757,7,14,Made Shot,Hook Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,36,36,1,1
6,"Millsap, Paul",200794,2.8,4,1,-8,7:17,21400501,21400501,H,"JAN 03, 2015 - POR vs. ATL",4,4,2,2,10.4,8.4,19,made,3.9,L,Shot Chart Detail,444,200746,200746,LaMarcus Aldridge,1610612757,7,14,Made Shot,Hook Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,36,36,1,1
7,"Millsap, Paul",200794,2.8,4,1,-8,7:17,21400501,21400501,H,"JAN 03, 2015 - POR vs. ATL",4,4,2,2,10.4,8.4,19,made,3.9,L,Shot Chart Detail,444,200746,200746,LaMarcus Aldridge,1610612757,7,14,Made Shot,Hook Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,36,36,1,1
8,"Antetokounmpo, Giannis",203507,2.6,2,0,-8,0:09,21400792,21400792,A,"FEB 11, 2015 - SAC @ MIL",4,4,0,3,NaN,25.7,12,missed,2.0,L,Shot Chart Detail,553,200752,200752,Rudy Gay,1610612758,0,7,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,27,-164,220,1,0
9,"Antetokounmpo, Giannis",203507,2.6,2,0,-8,0:09,21400792,21400792,A,"FEB 11, 2015 - SAC @ MIL",4,4,0,3,NaN,25.7,12,missed,2.0,L,Shot Chart Detail,553,200752,200752,Rudy Gay,1610612758,0,7,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,27,-164,220,1,0


In [3]:
hi=nbastats.PlayByPlay("0021400070")
hi.pbp()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0021400070,0,12,0,1,10:46 PM,12:00,None,None,None,None,None
1,0021400070,1,10,0,1,10:46 PM,12:00,Jump Ball Lopez vs. Chandler: Tip to Parsons,None,None,None,None
2,0021400070,2,2,1,1,10:46 PM,11:40,None,None,MISS Nowitzki 22' Jump Shot,None,None
3,0021400070,3,4,0,1,10:46 PM,11:40,Lillard REBOUND (Off:0 Def:1),None,None,None,None
4,0021400070,4,2,1,1,10:46 PM,11:25,MISS Batum 18' Jump Shot,None,None,None,None
5,0021400070,5,4,0,1,10:46 PM,11:24,None,None,Nelson REBOUND (Off:0 Def:1),None,None
6,0021400070,6,1,84,1,10:47 PM,11:14,None,None,Parsons 5' Running Bank Shot (2 PTS) (Nowitzki...,2 - 0,-2
7,0021400070,7,1,75,1,10:47 PM,10:56,Matthews 3' Driving Finger Roll Layup (2 PTS),None,None,2 - 2,TIE
8,0021400070,8,1,46,1,10:47 PM,10:37,None,None,Ellis 4' Running Jump Shot (2 PTS),4 - 2,-2
9,0021400070,9,2,1,1,10:48 PM,10:11,MISS Matthews 24' 3PT Jump Shot,None,None,None,None


In [39]:
!pip install py-goldsberry

    100% |████████████████████████████████| 1.4MB 148kB/s 
  Running setup.py bdist_wheel for py-goldsberry
  Stored in directory: /Users/harrisonchase/Library/Caches/pip/wheels/0f/f3/b2/2434606f2a8716f8e2210a63b156e230bda7c074858cc9460e
Successfully built py-goldsberry


In [4]:
import goldsberry
import pandas as pd

In [26]:
playersCurrent = pd.DataFrame(goldsberry.PlayerList(2014))
playersCurrent.head(10)

[{u'DISPLAY_LAST_COMMA_FIRST': u'Allen, Ray',
  u'FROM_YEAR': u'1996',
  u'PERSON_ID': 951,
  u'PLAYERCODE': u'ray_allen',
  u'ROSTERSTATUS': 1,
  u'TEAM_ABBREVIATION': u'MIA',
  u'TEAM_CITY': u'Miami',
  u'TEAM_CODE': u'heat',
  u'TEAM_ID': 1610612748,
  u'TEAM_NAME': u'Heat',
  u'TO_YEAR': u'2013'},
 {u'DISPLAY_LAST_COMMA_FIRST': u'Anderson, James',
  u'FROM_YEAR': u'2010',
  u'PERSON_ID': 202341,
  u'PLAYERCODE': u'james_anderson',
  u'ROSTERSTATUS': 1,
  u'TEAM_ABBREVIATION': u'PHI',
  u'TEAM_CITY': u'Philadelphia',
  u'TEAM_CODE': u'sixers',
  u'TEAM_ID': 1610612755,
  u'TEAM_NAME': u'76ers',
  u'TO_YEAR': u'2015'},
 {u'DISPLAY_LAST_COMMA_FIRST': u'Armstrong, Hilton',
  u'FROM_YEAR': u'2006',
  u'PERSON_ID': 200756,
  u'PLAYERCODE': u'hilton_armstrong',
  u'ROSTERSTATUS': 1,
  u'TEAM_ABBREVIATION': u'GSW',
  u'TEAM_CITY': u'Golden State',
  u'TEAM_CODE': u'warriors',
  u'TEAM_ID': 1610612744,
  u'TEAM_NAME': u'Warriors',
  u'TO_YEAR': u'2013'},
 {u'DISPLAY_LAST_COMMA_FIRST': u'Ayo

In [11]:
import requests

In [9]:
def shot_log(year,player_id):
    api_param = {'DateFrom' : '',
                    'DateTo' : '',
                    'GameSegment': '',
                    'LastNGames': 0,
                    'LeagueID': "00",
                    'Location': '',
                    'Month': 0,
                    'OpponentTeamID': 0,
                    'Outcome': '',
                    'Period': 0,
                    'PlayerID': player_id,
                    'Season': year,
                    'SeasonSegment': '',
                    'SeasonType': "Regular Season",
                    'TeamID': 0,
                    'VsConference':'',
                    'VsDivision': ''
                               }
    pull=requests.get("http://stats.nba.com/stats/playerdashptshotlog?",params=api_param)
    headers = pull.json()['resultSets'][0]['headers']
    values = pull.json()['resultSets'][0]['rowSet']
    huh=pd.DataFrame([dict(zip(headers, value)) for value in values])
    return huh


In [293]:

all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in players2014.iterrows():
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2014-15")
    all_chart=all_chart.append(chart.shotchart())
    shots = shot_log("2014-15",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
all_shots.to_csv("all_shots_2014.csv")
all_chart.to_csv("all_chart_2014.csv")

In [295]:
all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in season2013.iterrows():
    shots = shot_log("2013-14",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2013-14")
    all_chart=all_chart.append(chart.shotchart())
all_shots.to_csv("all_shots_2013.csv")
all_chart.to_csv("all_chart_2013.csv")

In [75]:
def players(a):
    url = "http://stats.nba.com/stats/commonallplayers?"
    api_param = {'IsOnlyCurrentSeason':"0",
                          'LeagueID': '00',
                          'Season': '2015-16'}
    pull = requests.get(url, params=api_param)
    _headers = pull.json()['resultSets'][0]['headers']
    _values = pull.json()['resultSets'][0]['rowSet']
    season=pd.DataFrame([dict(zip(_headers, value)) for value in _values])
    season = season.query("FROM_YEAR <= @a and TO_YEAR >= @a")
    return season
players('2013')

,DISPLAY_LAST_COMMA_FIRST,FROM_YEAR,PERSON_ID,PLAYERCODE,ROSTERSTATUS,TEAM_ABBREVIATION,TEAM_CITY,TEAM_CODE,TEAM_ID,TEAM_NAME,TO_YEAR
13,"Acy, Quincy",2012,203112,quincy_acy,1,SAC,Sacramento,kings,1610612758,Kings,2015
19,"Adams, Steven",2013,203500,steven_adams,1,OKC,Oklahoma City,thunder,1610612760,Thunder,2015
22,"Adrien, Jeff",2010,202399,jeff_adrien,1,NOP,New Orleans,pelicans,1610612740,Pelicans,2015
23,"Afflalo, Arron",2007,201167,arron_afflalo,1,NYK,New York,knicks,1610612752,Knicks,2015
29,"Ajinca, Alexis",2008,201582,alexis_ajinca,1,NOP,New Orleans,pelicans,1610612740,Pelicans,2015
36,"Aldrich, Cole",2010,202332,cole_aldrich,1,LAC,Los Angeles,clippers,1610612746,Clippers,2015
37,"Aldridge, LaMarcus",2006,200746,lamarcus_aldridge,1,SAS,San Antonio,spurs,1610612759,Spurs,2015
48,"Allen, Lavoy",2011,202730,lavoy_allen,1,IND,Indiana,pacers,1610612754,Pacers,2015
52,"Allen, Ray",1996,951,ray_allen,0,,,,0,,2013
53,"Allen, Tony",2004,2754,tony_allen,1,MEM,Memphis,grizzlies,1610612763,Grizzlies,2015


In [345]:
joined_shots=pd.read_csv("joined_shots_2014.csv")
max(joined_shots.LOC_Y)
plt.scatter(joined_shots.LOC_X,joined_shots.LOC_Y)
plt.show()

In [350]:
huh=joined_shots[joined_shots.PTS_TYPE==2]
plt.scatter(huh.LOC_X,huh.LOC_Y)
plt.show()

In [359]:
from bs4 import BeautifulSoup

In [364]:
t1970=requests.get("http://www.fleaflicker.com/nfl/leaders?season=2014&week=1&statType=2&sortMode=2&position=256&tableOffset=40")
soup = BeautifulSoup((t1970.text), "html.parser")
rows = soup.find("table", attrs={"span": "player injurable"})
print rows

None
